In [ ]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import json

import folium
from geopy.geocoders import Nominatim
import geopandas as gpd
import fiona
from arcgis.gis import GIS
from folium.plugins import MarkerCluster
from geopy.distance import geodesic
import shapely.geometry


import dash_html_components as html
import dash_leaflet as dl
import dash_leaflet.express as dlx
import pandas as pd
from dash_extensions.javascript import assign
from dash import Dash
from dash_extensions.javascript import arrow_function

In [ ]:

housing = pd.read_csv('../Data/house_coordinates.csv')
housing = housing.head(100)

In [ ]:
housing.longitude.max()

In [ ]:
housing_geo = gpd.GeoDataFrame(
    housing, geometry=gpd.points_from_xy(housing.longitude, housing.latitude))

In [ ]:
housing_geo

In [ ]:
housing_geo.to_file('housing_geo.geojson', driver='GeoJSON')

In [ ]:
housing_json = housing_geo.to_json()

In [ ]:
import json
housing_json = json.loads(housing_json)

In [ ]:
pbf = geobuf.encode(housing_json) # GeoJSON or TopoJSON -> Geobuf string
my_json = geobuf.decode(pbf) # Geobuf string -> GeoJSON or TopoJSON
my_json

In [ ]:
gpd.GeoSeries([housing_geo]).to_json()

In [ ]:
json.dumps(shapely.geometry.mapping(housing_geo))

In [ ]:
gis_osm_pofw = gpd.read_file('../GeoJsonFiles/iowa-latest-free/gis_osm_pofw_free_1.shp')


In [ ]:
churches = gis_osm_pofw.to_dict('records')

In [ ]:
gis_osm_pofw.head()

THIS ONE WORKS!!

In [ ]:
app = JupyterDash(__name__)

markers = [dl.Marker(position=[41.75245, -92.00558]), dl.CircleMarker(center=[55, 10])]

markers = gis_osm_pofw.apply(
    lambda row: dl.Marker(position=[row['lat'], row['lon']]),
    axis=1
)

app.layout = html.Div(dl.Map([
    dl.TileLayer(),
    dl.LayersControl(
        [dl.Overlay(dl.LayerGroup(markers), name="markers", checked=True)]
    )
], 
    center=[41.978, -93.576], zoom=12, 
    style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"}
))

app.run_server(mode='inline')


In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div( children=[
    dl.Map([
    dl.TileLayer(),
      
    dl.GeoJSON(data=housing_json, id="hous")

], 
         center=[42.03, -93.64], zoom=12,
    style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"}
),
    html.Div(id="state"), html.Div(id="PID")
])

@app.callback(Output("PID", "children"), [Input("hous", "click_feature")])
def capital_click(feature):
    if feature is not None:
        return f"You clicked {feature['properties']['PID']}"

app.run_server(mode='inline')

In [ ]:
app = JupyterDash(__name__)


geojson_filter = assign(
    "function(feature, context){{return ['909176150'].includes(feature.properties.pid);}}")
app.layout = html.Div(children=[
    dl.Map([
    dl.TileLayer(),
    dl.LayersControl(
        [dl.Overlay(
            dl.LayerGroup(
                dl.GeoJSON(
                    data=housing_json, id="hous", cluster=True, zoomToBoundsOnClick=True)
            ), name = "Housing", checked = "True"
        ) #,
#          dl.Overlay(
#             dl.LayerGroup(
#                 dl.GeoJSON(
#                     data=housing_json, id="hous", cluster=True, zoomToBoundsOnClick=True)
#             ), name = "Housing", checked = "True"
#         )
        ]
     )
    ],
        center=[42.03, -93.64], zoom=12,
        style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"}
    ),
    html.Div(id="PIDs")
])


            
            #             dl.GeoJSON(
#                 data=housing, options=dict(
#                 filter=geojson_filter),
#                        id="housing")),
#                     name="Housing", checked=True)]
#          ])

@app.callback(Output("PIDs", "children"), [Input("hous", "click_feature")])
def house_click(feature):
    if feature is not None:
        return f"You clicked {feature['properties']['PID']}"


app.run_server(mode='inline')


In [ ]:

# Generate some in-memory data.
bermuda = dlx.dicts_to_geojson([dict(lat=32.299507, lon=-64.790337)])
biosfera = dlx.geojson_to_geobuf(dlx.dicts_to_geojson([dict(lat=29.015, lon=-118.271)]))
# Create example app.
app = Dash()
app.layout = html.Div([
    dl.Map(center=[39, -98], zoom=4, children=[
        dl.TileLayer(),
        dl.GeoJSON(data=bermuda),  # in-memory geojson (slowest option)
        dl.GeoJSON(data=biosfera, format="geobuf"),  # in-memory geobuf (smaller payload than geojson)
        dl.GeoJSON(url="/assets/us-state-capitals.json", id="capitals"),  # geojson resource (faster than in-memory)
        dl.GeoJSON(url="/assets/us-states.pbf", format="geobuf", id="states",
                   hoverStyle=arrow_function(dict(weight=5, color='#666', dashArray=''))),  # geobuf resource (fastest option)
    ], style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"}, id="map"),
    html.Div(id="state"), html.Div(id="capital")
])


@app.callback(Output("capital", "children"), [Input("capitals", "click_feature")])
def capital_click(feature):
    if feature is not None:
        return f"You clicked {feature['properties']['name']}"


@app.callback(Output("state", "children"), [Input("states", "hover_feature")])
def state_hover(feature):
    if feature is not None:
        return f"{feature['properties']['name']}"



app.run_server(mode='inline')

In [ ]:
geojson

In [ ]:
    
    #### Some HTLM for controls
#     html.Label([
#         "Religion",
#         dcc.Dropdown(
#             id='religion', clearable=False,
#             value='plasma', options=[
#                 {'label': c, 'value': c}
#                 for c in gis_osm_pofw.fclass.unique()
#             ])
#     ]),
#     html.Label([
#         "Code",
#         dcc.RangeSlider(
#             id='code',
#             min=gis_osm_pofw.code.min(),
#             max=gis_osm_pofw.code.max(),
#             step=0.5,
#             value=[3200, 3300]
#         ),
#     html.Div(id='output-container-range-slider')
#     ])


In [ ]:
app = JupyterDash(__name__)

app.layout = dl.Map(dl.TileLayer(), style={'width': '1000px', 'height': '500px'})
app.run_server(mode='inline')

In [ ]:
@app.callback(
[Output(component_id="out")])



### RANGE SLIDER
#### FOR HTML
#     html.Label([
#         "Price Range",
#         dcc.RangeSlider(
#             id='price',
#             min=gis_osm_pofw.price.min(),
#             max=gis_osm_pofw.price.max(),
#             step=0.5,
#             value=[100000, 150000]
#         ),
#     html.Div(id='price-range-slider')
#     ])

#### For callback
@app.callback(
    dash.dependencies.Output('price-range-slider', 'children'),
    [dash.dependencies.Input('pricer', 'value')])
def update_output(value):
    return 'You have selected "{}"'.format(value)


In [ ]:
cities = [dict(name="Aalborg", lat=57.0268172, lon=9.837735),
          dict(name="Aarhus", lat=56.1780842, lon=10.1119354),
          dict(name="Copenhagen", lat=55.6712474, lon=12.5237848)]


# Generate geojson with a marker for each city and name as tooltip.
geojson = dlx.dicts_to_geojson([{**c, **dict(tooltip=c['name'])} for c in churches])
# Create javascript function that filters out all cities but Aarhus.
geojson_filter = assign("function(feature, context){{return ['Aarhus'].includes(feature.properties.name);}}")
# Create example app.
app = Dash()
app.layout = html.Div([
    dl.Map(children=[
        dl.TileLayer(),
        dl.GeoJSON(data=geojson, options=dict(filter=geojson_filter), id="geojson")
    ], style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"}, id="map"),
])


In [ ]:
map = folium.Map(location = [42.0219, -93.6227], tiles='OpenStreetMap' , zoom_start = 12)
# Create a folium marker cluster
marker_cluster = MarkerCluster(locations = gis_osm_pofw['locations'],
                               popups=list(gis_osm_pofw['name']), 
                               name="Churches", show=False)

# Add marker cluster to map
marker_cluster.add_to(map)

folium.LayerControl().add_to(map)


# Show map
map

gis_osm_pofw

In [ ]:
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("JupyterDash Demo"),
    dcc.Graph(id='graph'),
    html.Label([
        "colorscale",
        dcc.Dropdown(
            id='colorscale-dropdown', clearable=False,
            value='plasma', options=[
                {'label': c, 'value': c}
                for c in px.colors.named_colorscales()
            ])
    ]),
])

In [ ]:
df = px.data.tips()
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("colorscale-dropdown", "value")]
)
def update_figure(colorscale):
    return px.scatter(
        df, x="total_bill", y="tip", color="size",
        color_continuous_scale=colorscale,
        render_mode="webgl", title="Tips"
    )
# Run app and display result inline in the notebook
app.run_server(mode='inline')

In [ ]:
# df = gpd.read_file('../GeoJsonFiles/iowa-latest-free/gis_osm_pofw_free_1.shp')
# df['lon'] = df['geometry'].x
# df['lat'] = df['geometry'].y
df 

In [ ]:

colorscale = ['red', 'yellow', 'green', 'blue', 'purple']  # rainbow
chroma = "https://cdnjs.cloudflare.com/ajax/libs/chroma-js/2.1.0/chroma.min.js"  # js lib used for colors
color_prop = 'code'

# Pre process the data into geobuf.
  # drop irrelevant columns

dicts = df.to_dict('rows')
for item in dicts:
    item["tooltip"] = "{} ({:.1f})".format(item['name'], item[color_prop])  # bind tooltip

geojson = dlx.dicts_to_geojson(dicts, lon="lon")  # convert to geojson
geobuf = dlx.geojson_to_geobuf(geojson)  # convert to geobuf
# Create a colorbar.
vmax = df[color_prop].max()
colorbar = dl.Colorbar(colorscale=colorscale, width=20, height=150, min=0, max=vmax, unit='/km2')
# Geojson rendering logic, must be JavaScript as it is executed in clientside.
point_to_layer = assign("""function(feature, latlng, context){
    const {min, max, colorscale, circleOptions, colorProp} = context.props.hideout;
    const csc = chroma.scale(colorscale).domain([min, max]);  // chroma lib to construct colorscale
    circleOptions.fillColor = csc(feature.properties[colorProp]);  // set color based on color prop.
    return L.circleMarker(latlng, circleOptions);  // sender a simple circle marker.
}""")
cluster_to_layer = assign("""function(feature, latlng, index, context){
    const {min, max, colorscale, circleOptions, colorProp} = context.props.hideout;
    const csc = chroma.scale(colorscale).domain([min, max]);
    // Set color based on mean value of leaves.
    const leaves = index.getLeaves(feature.properties.cluster_id);
    let valueSum = 0;
    for (let i = 0; i < leaves.length; ++i) {
        valueSum += leaves[i].properties[colorProp]
    }
    const valueMean = valueSum / leaves.length;
    // Render a circle with the number of leaves written in the center.
    const icon = L.divIcon.scatter({
        html: '<div style="background-color:white;"><span>' + feature.properties.point_count_abbreviated + '</span></div>',
        className: "marker-cluster",
        iconSize: L.point(40, 40),
        color: csc(valueMean)
    });
    return L.marker(latlng, {icon : icon})
}""")
# Create geojson.
geojson = dl.GeoJSON(data=geobuf, id="geojson", format="geobuf",
                     zoomToBounds=True,  # when true, zooms to bounds when data changes
                     cluster=True,  # when true, data are clustered
                     clusterToLayer=cluster_to_layer,  # how to draw clusters
                     zoomToBoundsOnClick=True,  # when true, zooms to bounds of feature (e.g. cluster) on click
                     options=dict(pointToLayer=point_to_layer),  # how to draw points
                     superClusterOptions=dict(radius=150),   # adjust cluster size
                     hideout=dict(colorProp=color_prop, circleOptions=dict(fillOpacity=1, stroke=False, radius=5),
                                  min=0, max=vmax, colorscale=colorscale))
# Create the app.
app = Dash(external_scripts=[chroma], prevent_initial_callbacks=True)
app.layout = html.Div([
    dl.Map([dl.TileLayer(), geojson, colorbar]),
], style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block", "position": "relative"})

if __name__ == '__main__':
    app.run_server(mode='inline')